# 7.3 Getting the most out of your models
In this section we will talk about what some of the best methods for achieving excellent results with machine learning and deep learning models.

## 7.3.1 Advanced architecture patterns
A few of the most important design patterns are residual connections (discussed in section 7.1), normalization, and depthwise separable convolution. These patterns are especially useful when building high-performing convnets, but they are found in other architectures as well.

**BATCH NORMALIZATION**

*Normalization* is a broad categoriy of methods that seek to make different samples seen by a machine learning model more similar to each other, which enables the model to learn and generalize well to new data. The most common form of data normalization is centering the data on 0 by subtracting the mean from the data, and giving the data a unit standard deviation by dividing the data by its standard deviation. This makes the assumption that the data follows a normal (or Gaussian) distribution and makes sure the distribution is centered and scaled to unit variance:

`normalized_data = (data - np.mean(data, axis=...)) / np.std(data, axis=...)`

Batch normalization can adaptively normalize data even as the mean and variance change over time during training. It works by internally maintaining an exponential moving average of the batch-wise mean and variance of the data seen during training. This mainly helps with gradient propagation and allows for deeper networks.

The `BatchNormalization` layer is typically used after a convolutional or densely connected layer:

`conv_model.add(layers.Conv2D(32, 3, activation='relu'))` <-- After Conv layer
`conv_model.add(layers.BatchNormalization())`

`dense_model.add(layers.Dense(32, activation='relu'))` <-- After Dense layer
`dense_model.add(layers.BatchNormalization())`

The `BatchNormalization` layer takes an `axis` argument, which specifies the feature axis that should be normalized. The default of this argument is -1, the last axis in the input tensor. This is the correct value when using `Dense, Conv1D, RNN` and `Conv2D` (where `data_format` is set to "`channels_last`) layers. In the case of `Conv2D` with `data_format` set to "`channels_first`, the features axis is 1.

**DEPTHWISE SEPARABLE CONVOLUTION**

The **depthwise separable convolution** (`SeparableConv2D`) layer can be used as a replacement for a `Conv2D` that will make a model lighter, faster, and cause it to perform a few percentage points better on its task. This layer essentially separates the learning of spatial features and the learning of channel-wise features, which makes a lot of sense if you assum that spatial locations in the input are highly correlated, but different channels are fairly independent. It requires significantly fewer parameters and involves fewer computations, making models smaller and faster.

![separable](images/7_3_1_separable.jpg)

These advantages become especially important when we're training small models from scratch on limited data. For example, here's how we can build a lightweight, depthwise separable convnet for an image-classification task on a small dataset:

In [ ]:
from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3,
                                 activation='relu',
                                 input_shape=(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

When it comes to larger-scale models, depthwise separable convolutions are the basis of the Xception architecture, a high-performing convnet that comes packaged with Keras.

## 7.3.2 Hyperparameter optimization
When building a deep-learning model, you have to make many seemingly arbitrary decisions: How many layers to stack? How many units or filters should go in each layer? Should you use `relu` as activation, or another function? Should you use `BatchNormalization` after a given layer? How much dropout should you use? All of these architecture-level parameters are called **hyperparameters** to distinguish them from the parameters of a model, which are trained via backpropagation.

Your initial decisions for the hyperparameters are almost always suboptimal. You can refine your choices and retrain your model over and over again, but this is still an arduous process. Instead of making all these tweaks by hand, it's better to leave this to a machine, leading us to the field of automatic hyperparameter optimization. The process of optimizing hyperparameters typically looks like this:
 
 1. Choose a set of hyperparameters (automatically).
 2. Build the corresponding model.
 3. Fit it to your training data, and measure the final performance on the validation data.
 4. Choose the next set of hyperparameters to try (automatically).
 5. Repeat.
 6. Eventually, measure performance on your test data.
 
The key to this process is the algorithm that uses this history of validation performance, given various sets of hyperparameters, to choose the next set of hyperparameters to evaluate. Training the weights of a model is relatively easy: you compute a loss function on a mini-batch of data and then use the Backpropagation algorithm to move the weights in the right direction. Updating hyperparameters, on the other hand, is extremely challenging. Consider the following:

 - Computing the feedback signal (does this set of hyperparameters lead to a high-performing model on this task?) can be extremely expensive: it requires creating and training a new model from scratch on your dataset.
 - The hyperparameter space is typically made of discrete decisions and this isn't continuous of differentiable. You typically can't do gradient descent in hyperparameter space. Instead, you must rely on gradient-free optimization techniques, which naturally are far less efficient than gradient descent.
 
Often, it turns out that random search (choosing hyperparameters to evaluate at random, repeatedly) is the best solution, despite being so naive. One tool that is helpful is called Hyperopt, which is a Python library for hyperparameter optimization that internally uses trees of Parzen estimators to predict sets of hyperparameters that are likely to work well. 

## 7.3.3 Model ensembling
Another powerful technique for obtaining the best possible results on a task is **model ensembling**. Ensembling consists of pooling together the predictions of a set of different models, to produce better predictions. 

Ensembling relies on the assumption that different good models trained independently are likely to be good for different reasons: each model looks at slightly different aspects of the data to make its predictions, getting part of the "truth" but not all of it. Let's use classification as an example. The easier way to pool the predictions of a set of classifiers (to ensemble the classifiers) is to average their predictions at inference time:

**USE FOUR DIFFERENT MODELS TO COMPUTE INITIAL PREDICTIONS**

In [ ]:
preds_a = model_a.predict(x_val)                              
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)

final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d)

This will only work if the classifiers are more or less equally good. If one of them is significantly worst than the others, the final predictions may not be as good as the best classifier of the group.

A smarter way to ensemble classifiers is to do a weighted average, where the weights are learned on the validation data - typically, the better classifiers are given a higher weight, and the worse classifiers are given a lower weight. To search for a good set of ensembling weights, we can use random search or a simple optimization algorithm such as Nelder-Mead:

In [ ]:
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)

final_preds = 0.5 * preds_a + 0.25 * preds_b + 0.1 * preds_c + 0.15 * preds_d 1

There are many possible variants: we can do an average of an exponential of the predictions, for instance. In general, a simple weighted average with weights optimized on the validation data provides a very strong baseline.

The key to making ensembling work is the diversity of the set of classifiers. Diversity is strength and is what makes ensembling work. If all of our models are biased in different ways, the biases will cancel each other out, and the ensemble will be more robust and more accurate.

For this reason, we should ensemble models that are *as good as possible* while being *as different as possible*. One thing that is largely not worth doing is ensembling the same network trained several times independently, from different random initializations. This would lead to an ensemble that lacks diversity and will provide only a tiny improvement over any single model.

One thing that works well in practice (but doesn't generalize to every problem domain) is the use of an ensemble of tree-based methods (random forests or gradient-boosted trees) and deep neural networks. It's not so much about how good your best model is; it's about the diversity of your set of candidate models.

One style of basic ensemble that has been very successful is the **wide and deep** category of models, blending deep learning with shallow learning. 

## 7.3.4 Wrapping up
 - When building high-performing deep convnets, you’ll need to use residual connections, batch normalization, and depthwise separable convolutions. In the future, it’s likely that depthwise separable convolutions will completely replace regular convolutions, whether for 1D, 2D, or 3D applications, due to their higher representational efficiency.
 - Building deep networks requires making many small hyperparameter and architecture choices, which together define how good your model will be. Rather than basing these choices on intuition or random chance, it’s better to systematically search hyperparameter space to find optimal choices. At this time, the process is expensive, and the tools to do it aren’t very good. But the Hyperopt and Hyperas libraries may be able to help you. When doing hyperparameter optimization, be mindful of validation-set overfitting!
 - Winning machine-learning competitions or otherwise obtaining the best possible results on a task can only be done with large ensembles of models. Ensembling via a well-optimized weighted average is usually good enough. Remember: diversity is strength. It’s largely pointless to ensemble very similar models; the best ensembles are sets of models that are as dissimilar as possible (while having as much predictive power as possible, naturally).